<a href="https://colab.research.google.com/github/tatsubo777/Kaggle_Signate_competition/blob/master/predict_feature_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaggle

In [3]:
!chmod 600 /root/.kaggle/kaggle.json


In [4]:
'''
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

SyntaxError: ignored

In [ ]:
'''!kaggle competitions list

In [ ]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

In [ ]:
'''!unzip competitive-data-science-predict-future-sales.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#結果提出

!kaggle competitions submit titanic -f my_submission.csv -m "comment"

In [5]:
import numpy as np
import pandas as pd

In [6]:
train = pd.read_csv("/content/sales_train.csv")
test = pd.read_csv("/content/test.csv")

print(train.shape)
train.head()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


item_cnt_day→商品の販売数を示す
予測するのはitem_cnt_dayのshopごとかつitemごとの翌月の総数

In [7]:
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
test.head()

FileNotFoundError: ignored

In [9]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [11]:
train['date'][0]

'02.01.2013'

In [12]:
train['date'][0][:2]

'02'

In [13]:
train['date'].apply(lambda x: x[:2])

0          02
1          03
2          05
3          06
4          15
           ..
2935844    10
2935845    09
2935846    14
2935847    22
2935848    03
Name: date, Length: 2935849, dtype: object

In [14]:
train['day'] = train['date'].apply(lambda x: int(x[:2]))
train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day
0,02.01.2013,0,59,22154,999.00,1.0,2
1,03.01.2013,0,25,2552,899.00,1.0,3
2,05.01.2013,0,25,2552,899.00,-1.0,5
3,06.01.2013,0,25,2554,1709.05,1.0,6
4,15.01.2013,0,25,2555,1099.00,1.0,15
5,10.01.2013,0,25,2564,349.00,1.0,10
6,02.01.2013,0,25,2565,549.00,1.0,2
7,04.01.2013,0,25,2572,239.00,1.0,4
8,11.01.2013,0,25,2572,299.00,1.0,11
9,03.01.2013,0,25,2573,299.00,3.0,3


In [15]:
train['day'].describe()

count    2.935849e+06
mean     1.585267e+01
std      8.923483e+00
min      1.000000e+00
25%      8.000000e+00
50%      1.600000e+01
75%      2.400000e+01
max      3.100000e+01
Name: day, dtype: float64

In [16]:
train['month'] = train['date'].apply(lambda x: int(x[3:5]))
train.head(5)
                                     

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month
0,02.01.2013,0,59,22154,999.00,1.0,2,1
1,03.01.2013,0,25,2552,899.00,1.0,3,1
2,05.01.2013,0,25,2552,899.00,-1.0,5,1
3,06.01.2013,0,25,2554,1709.05,1.0,6,1
4,15.01.2013,0,25,2555,1099.00,1.0,15,1


In [17]:
train['month'].describe()

count    2.935849e+06
mean     6.247717e+00
std      3.536219e+00
min      1.000000e+00
25%      3.000000e+00
50%      6.000000e+00
75%      9.000000e+00
max      1.200000e+01
Name: month, dtype: float64

In [18]:
train['year'] = train['date'].apply(lambda x: int(x[6:]))
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,02.01.2013,0,59,22154,999.00,1.0,2,1,2013
1,03.01.2013,0,25,2552,899.00,1.0,3,1,2013
2,05.01.2013,0,25,2552,899.00,-1.0,5,1,2013
3,06.01.2013,0,25,2554,1709.05,1.0,6,1,2013
4,15.01.2013,0,25,2555,1099.00,1.0,15,1,2013


In [19]:
train['year'].describe()

count    2.935849e+06
mean     2.013777e+03
std      7.684790e-01
min      2.013000e+03
25%      2.013000e+03
50%      2.014000e+03
75%      2.014000e+03
max      2.015000e+03
Name: year, dtype: float64

In [20]:
train.drop(columns ='date',inplace=True)
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,0,59,22154,999.00,1.0,2,1,2013
1,0,25,2552,899.00,1.0,3,1,2013
2,0,25,2552,899.00,-1.0,5,1,2013
3,0,25,2554,1709.05,1.0,6,1,2013
4,0,25,2555,1099.00,1.0,15,1,2013


In [21]:
print(train['year'].max())

2015


In [22]:
train_2015 = train.query('year == 2015',engine='python')
print(train_2015.shape)

(612426, 8)


In [23]:
print(train_2015['month'].max())

10


In [24]:
print(train_2015.loc[train_2015['month'] == 10]['day'].max())

31


2015年11月から予測すればいい

In [25]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [26]:
test['year'] = 2015
test['month'] = 11
test.head()

,ID,shop_id,item_id,year,month
0,0,5,5037,2015,11
1,1,5,5320,2015,11
2,2,5,5233,2015,11
3,3,5,5232,2015,11
4,4,5,5268,2015,11


In [27]:
train['date_block_num'].describe()

count    2.935849e+06
mean     1.456991e+01
std      9.422988e+00
min      0.000000e+00
25%      7.000000e+00
50%      1.400000e+01
75%      2.300000e+01
max      3.300000e+01
Name: date_block_num, dtype: float64

In [28]:
train.loc[train['date_block_num'] == 0]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,0,59,22154,999.00,1.0,2,1,2013
1,0,25,2552,899.00,1.0,3,1,2013
2,0,25,2552,899.00,-1.0,5,1,2013
3,0,25,2554,1709.05,1.0,6,1,2013
4,0,25,2555,1099.00,1.0,15,1,2013
...,...,...,...,...,...,...,...,...
115685,0,46,32,349.00,1.0,19,1,2013
115686,0,46,32,349.00,1.0,18,1,2013
115687,0,46,35,399.00,2.0,26,1,2013
115688,0,46,621,149.00,1.0,31,1,2013


In [29]:
train.loc[train['date_block_num'] == 1]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
115690,1,50,3880,1499.0,1.0,21,2,2013
115691,1,50,3880,1499.0,1.0,14,2,2013
115692,1,50,4128,899.0,1.0,21,2,2013
115693,1,50,4124,249.0,1.0,13,2,2013
115694,1,50,3880,1499.0,1.0,24,2,2013
...,...,...,...,...,...,...,...,...
224298,1,28,13511,1599.0,2.0,23,2,2013
224299,1,28,13511,1599.0,1.0,26,2,2013
224300,1,28,13413,1999.0,1.0,18,2,2013
224301,1,28,13413,1999.0,1.0,20,2,2013


In [30]:
train.loc[train['date_block_num'] == 33]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
2882335,33,45,13315,649.0,1.0,23,10,2015
2882336,33,45,13880,229.0,1.0,5,10,2015
2882337,33,45,13881,659.0,1.0,2,10,2015
2882338,33,45,13881,659.0,1.0,12,10,2015
2882339,33,45,13923,169.0,1.0,4,10,2015
...,...,...,...,...,...,...,...,...
2935844,33,25,7409,299.0,1.0,10,10,2015
2935845,33,25,7460,299.0,1.0,9,10,2015
2935846,33,25,7459,349.0,1.0,14,10,2015
2935847,33,25,7440,299.0,1.0,22,10,2015


In [31]:
test['date_block_num'] = 34
test.head()

,ID,shop_id,item_id,year,month,date_block_num
0,0,5,5037,2015,11,34
1,1,5,5320,2015,11,34
2,2,5,5233,2015,11,34
3,3,5,5232,2015,11,34
4,4,5,5268,2015,11,34


店舗コードが0でitem_idが32の商品は2013年3月以降全く売れていないことがわかる。

In [32]:
train.loc[(train['shop_id'] == 0) & (train['item_id'] == 32)]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
40085,0,0,32,221.0,2.0,3,1,2013
40086,0,0,32,221.0,2.0,21,1,2013
40087,0,0,32,221.0,1.0,25,1,2013
40088,0,0,32,221.0,1.0,31,1,2013
173447,1,0,32,221.0,1.0,25,2,2013
173448,1,0,32,221.0,2.0,22,2,2013
173449,1,0,32,221.0,2.0,16,2,2013
173450,1,0,32,221.0,1.0,15,2,2013
173452,1,0,32,221.0,2.0,12,2,2013
173453,1,0,32,221.0,1.0,5,2,2013


In [33]:
train.loc[(train['shop_id'] == 59) & (train['item_id'] == 22162) ]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
2435401,25,59,22162,399.0,1.0,9,2,2015
2435402,25,59,22162,399.0,1.0,13,2,2015
2435403,25,59,22162,399.0,1.0,14,2,2015
2435404,25,59,22162,399.0,1.0,15,2,2015
2435405,25,59,22162,399.0,1.0,16,2,2015
2435406,25,59,22162,399.0,1.0,17,2,2015
2435407,25,59,22162,399.0,1.0,19,2,2015
2435408,25,59,22162,399.0,1.0,22,2,2015
2435409,25,59,22162,399.0,1.0,24,2,2015
2507082,26,59,22162,399.0,1.0,1,3,2015


6月7月が売り上げ0の行を作る

In [34]:
shops = train['shop_id'].unique()
items = train['item_id'].unique()

前年月における全店舗の商品データを作ります。
.uniqueで全ての基準を取得できる

In [35]:
from tqdm.notebook import trange
all_data = []
for i in trange(34):
    for shop in shops:
        for item in items:
            all_data.append([i,shop,item])
all_data = pd.DataFrame(all_data,columns = ['date_block_num','shop_id','item_id'])

  0%|          | 0/34 [00:00<?, ?it/s]

学習データの"date_block_num"は０～33です。

全ての"shop_id"は.uniqueで取得しました。"item_id"も同じです。

これらをリストにして行方向に次々と積み上げていきましょう。

trangeはプログレスバーを作るコードです。

実行するとall_dataとして全年月日における全店舗の全商品が網羅されたデータが作られます。

In [36]:
print(all_data.shape)
print(train.shape)
all_data.head()

(44486280, 3)
(2935849, 8)


,date_block_num,shop_id,item_id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564


月の売り上げ総数を計算

In [37]:
train.groupby(["date_block_num","shop_id","item_id"], as_index= False).item_cnt_day.sum()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1609119,33,59,22087,6.0
1609120,33,59,22088,2.0
1609121,33,59,22091,1.0
1609122,33,59,22100,1.0


In [38]:
train[(train['date_block_num']==0) & (train['shop_id'] == 0) & (train['item_id']==32)]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
40085,0,0,32,221.0,2.0,3,1,2013
40086,0,0,32,221.0,2.0,21,1,2013
40087,0,0,32,221.0,1.0,25,1,2013
40088,0,0,32,221.0,1.0,31,1,2013


In [39]:
all_data = all_data.merge(train.groupby(['date_block_num','shop_id','item_id'],as_index=False)['item_cnt_day'].sum(),
                         on=['date_block_num','shop_id','item_id'],
                         how='left')
all_data.head(5)

,date_block_num,shop_id,item_id,item_cnt_day
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN


pandasの.mergeでデータを結合しました。

onはどの列を参照して結合するか指定します。howを"left"にすると元のall_dataに存在しないカテゴリは無視されます。

NaNになっている部分は販売数が０です。

In [40]:
all_data.rename(columns={'item_cnt_day' : 'item_cnt_month'},inplace=True)
all_data.fillna(0,inplace=True)
all_data.head(5)

,date_block_num,shop_id,item_id,item_cnt_month
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0


In [41]:
all_data['year'] = 0
all_data['month'] = 0
year = 2013
month = 1
for date_block_num in range(34):
    all_data.loc[all_data['date_block_num'] == date_block_num, "year"] = year
    all_data.loc[all_data['date_block_num'] == date_block_num, "month"] = month
    month += 1
    if month == 13:
        year += 1
        month = 1

年月がなくなったので新しく作成

In [42]:
all_data.head(5)

,date_block_num,shop_id,item_id,item_cnt_month,year,month
0,0,59,22154,1.0,2013,1
1,0,59,2552,0.0,2013,1
2,0,59,2554,0.0,2013,1
3,0,59,2555,0.0,2013,1
4,0,59,2564,0.0,2013,1


In [43]:
all_data.tail()

,date_block_num,shop_id,item_id,item_cnt_month,year,month
44486275,33,36,8428,0.0,2015,10
44486276,33,36,7903,0.0,2015,10
44486277,33,36,7610,0.0,2015,10
44486278,33,36,7635,0.0,2015,10
44486279,33,36,7640,0.0,2015,10


モデル作成

In [44]:
all_data.head()

,date_block_num,shop_id,item_id,item_cnt_month,year,month
0,0,59,22154,1.0,2013,1
1,0,59,2552,0.0,2013,1
2,0,59,2554,0.0,2013,1
3,0,59,2555,0.0,2013,1
4,0,59,2564,0.0,2013,1


In [45]:
test.head()

,ID,shop_id,item_id,year,month,date_block_num
0,0,5,5037,2015,11,34
1,1,5,5320,2015,11,34
2,2,5,5233,2015,11,34
3,3,5,5232,2015,11,34
4,4,5,5268,2015,11,34


all_dataとtestの列を確認します。'ID'は使わないので他の列があればOK

In [46]:
import gc
train = all_data.loc[all_data["date_block_num"] != 33]
valid = all_data.loc[all_data['date_block_num'] == 33]

del all_data; gc.collect()

18

学習データと評価データに分けました。

分割するとall_dataは不要なのでdelで消してgc.collectでメモリを整理しましょう。

In [47]:
import lightgbm as lgb

In [48]:
params = {
    "objective" : "regression",
    "metric" : "rmse",
    "verbosity" : -1

}

学習パラメータを設定します。

objective：モデルの種類。regressionは回帰です。

metric：評価指標。rmseは平均二乗誤差の平方根です。

verbosity：出力形態。設定しなくてもOK。

In [49]:
use_cols = ["date_block_num","shop_id","item_id","year","month"]
target = "item_cnt_month"

X_train = train[use_cols]
y_train = train[target]
X_valid = valid[use_cols]
y_valid= valid[target]

学習に使う列名をリスト化しました。

Xは学習に使う列でyは予測したい列です。

In [50]:
train_set = lgb.Dataset(X_train,y_train)
valid_set = lgb.Dataset(X_valid,y_valid)
del train, valid; gc.collect()

150

.DatasetでLightGBM専用の形式に変換します。

In [51]:
model = lgb.train(
    params = params,train_set = train_set,valid_sets = [valid_set],num_boost_round = 1000,early_stopping_rounds = 10
)

[1]	valid_0's rmse: 2.28237
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 2.28214
[3]	valid_0's rmse: 2.28196
[4]	valid_0's rmse: 2.28186
[5]	valid_0's rmse: 2.2818
[6]	valid_0's rmse: 2.28165
[7]	valid_0's rmse: 2.28163
[8]	valid_0's rmse: 2.28156
[9]	valid_0's rmse: 2.28154
[10]	valid_0's rmse: 2.28153
[11]	valid_0's rmse: 2.28147
[12]	valid_0's rmse: 2.28146
[13]	valid_0's rmse: 2.28144
[14]	valid_0's rmse: 2.28132
[15]	valid_0's rmse: 2.28129
[16]	valid_0's rmse: 2.28124
[17]	valid_0's rmse: 2.28124
[18]	valid_0's rmse: 2.28074
[19]	valid_0's rmse: 2.28066
[20]	valid_0's rmse: 2.28065
[21]	valid_0's rmse: 2.28059
[22]	valid_0's rmse: 2.27992
[23]	valid_0's rmse: 2.2799
[24]	valid_0's rmse: 2.2799
[25]	valid_0's rmse: 2.27989
[26]	valid_0's rmse: 2.27986
[27]	valid_0's rmse: 2.2798
[28]	valid_0's rmse: 2.2798
[29]	valid_0's rmse: 2.27978
[30]	valid_0's rmse: 2.27978
[31]	valid_0's rmse: 2.2802
[32]	valid_0's rmse: 2.28019
[33]	valid_0's rmse: 2.28

In [52]:
X_test = test[use_cols]
preds = model.predict(X_test)

In [53]:
submit = pd.DataFrame()
submit['ID'] = test['ID']
submit[target] = preds
submit.head()

,ID,item_cnt_month
0,0,0.06513
1,1,0.06513
2,2,0.06513
3,3,0.06513
4,4,0.06513


.DataFrameでカラのデータセットを作りました。

"ID"はtestのものをそのまま使いましょう。

"item_cnt_month"はさっき予測したデータです。

In [55]:
submit.to_csv("submission.csv", index = False)

In [56]:
#結果提出

!kaggle competitions submit competitive-data-science-predict-future-sales  -f submission.csv -m "comment"

100% 5.44M/5.44M [00:00<00:00, 22.3MB/s]
Successfully submitted to Predict Future Sales